<h1><center>BATTLE OF THE NEIGHBORHOODS -IBM PROJECT</center><h1>

INTRODUCTION /BUSSINESS PROBLEM


INTRODUCTION: This is written to help a European based courier company looking gain entrant into North America beginning with Canada. They are headquartered in the United Kingdom and have presence in most part of Europe. Their services are segmented into freight and courier. 
To penetrate the Canadian market, they are undecided between leading with either Toronto or Calgary, as jointly, these cities command the biggest market in Canada. Due to limited knowledge of this market, they have contacted us to provide information that will help the selection of a best-case city and possibly a preferred city for operations.


Our engagement with them shows that the following criteria will be key to deciding a preferred city;
<ul>
<li>	City with the best base brackete tax rate</li>
<li>	Best contributor to national GDP   </li>
<li>	Above average GDP   </li>
<li>	Average to above average total population</li>

</ul>

These criteria have been provided on the back of indices that have been important to their success in their current market.
Our Objective is to locate neighborhoods that meet the criteria and recommend a best-case to the company and the target audience. The information received from this study will help the company decide on a city to commence their operations by providing required data about a host of cities.
Also, the information received would also help other same kind businesses considering the Canadian courier market.


<u>DATA DESCRIPTION


The information needed by the client will be obtained from the following sources;

<ol>
    
<li>	<a href="https://en.m.wikipedia.org/wiki/Income_taxes_in_Canada">Canada Income tax</a> & <a href="https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html">provincial tax by Wikipedia</a>: This provides specific details about personal income and corporate tax. The information is provided by the Federal Tax Rates 2020 and Wikipedia. Wikeipedia referenced the following sources; Canada Revenue Agency,2018 Federal Personal Income Tax Rates, Corporate Tax Rate and the Fiscal Reference Tables October 2018.The information collated include tax bracket per each province for 2020. This will be useful in comparing the tax requirement of Ontario and Alberta.</li> 



<li>	<a href="https://en.m.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_gross_domestic_product">Canada GDP contribution by Wikipedia</a>: This information originated from statistics Canada and the department of Finance Canada. The information shows us the percentage contribution as well absolute figure of the GDP contribution of each province. </li>

<li>	 <a href="https://en.m.wikipedia.org/wiki/Population_of_Canada_by_province_and_territory">Population per province and city</a>: The information provided was on the back of the 2016 census and the Interprovincial migration. This help us determine the average population of cities and decide on those that meet our criteria</li> 

<li>	<a href="https://en.m.wikipedia.org/wiki/National_Highway_System_(Canada)">Road network</a>: This shows us the size of the road network in a city. This information will help determine the accessibility of a city. This will be important in the courier and freight business.</li>

<li>	<a href="https://developer.foursquare.com/">Foursquare API</a> : To collect location data about competitors in the preferred location</li>
    </ol>


In [94]:
# importing necessary libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
##import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width',1000)

print('Libraries imported.')

Libraries imported.


In [95]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])


In [96]:
url='https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html'
result=requests.get(url)
soup=BeautifulSoup(result.content,'html.parser')
table=soup.find('table')
trs=table.find_all('tr')
rows=[]

for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
taxtable = []
for row in rows:
    provinces = row[0].text.rstrip()
    taxrates = row[1].text.rstrip() 
    taxtable.append([provinces,taxrates])
    
taxtable

[['Newfoundland and Labrador',
  '8.7% on the first $37,929 of taxable income, +\n14.5% on the next $37,929, +\n15.8% on the next $59,574, +\n17.3% on the next $54,172, +\n18.3% on the amount over $189,604'],
 ['Prince Edward Island',
  '9.8% on the first $31,984 of taxable income, +\n13.8% on the next $31,985, +\n16.7% on the amount over $63,969'],
 ['Nova Scotia',
  '8.79% on the first $29,590 of taxable income, +\n14.95% on the next $29,590, +\n16.67% on the next $33,820, +\n17.5% on the next $57,000, +\n21% on the amount over $150,000'],
 ['New Brunswick',
  '9.68% on the first $43,401 of taxable income, +\n14.82% on the next $43,402, +\n16.52% on the next $54,319, +\n17.84% on the next $19,654, +\n20.3% on the amount over $160,776'],
 ['Quebec', 'Go to Income tax rates (Revenu Québec Web site).'],
 ['Ontario',
  '5.05% on the first $44,740 of taxable income, +\n9.15% on the next $44,742, +\n11.16% on the next $60,518, +\n12.16% on the next $70,000, +\n13.16 % on the amount over $2

In [98]:
cols = ['provinces','taxratesCode',]
df_tax = pd.DataFrame(taxtable, columns=cols)
print(df_tax.shape)

(13, 2)


In [99]:
df_tax

,provinces,taxratesCode
0,Newfoundland and Labrador,"8.7% on the first $37,929 of taxable income, +..."
1,Prince Edward Island,"9.8% on the first $31,984 of taxable income, +..."
2,Nova Scotia,"8.79% on the first $29,590 of taxable income, ..."
3,New Brunswick,"9.68% on the first $43,401 of taxable income, ..."
4,Quebec,Go to Income tax rates (Revenu Québec Web site).
5,Ontario,"5.05% on the first $44,740 of taxable income, ..."
6,Manitoba,"10.8% on the first $33,389 of taxable income, ..."
7,Saskatchewan,"10.5% on the first $45,225 of taxable income, ..."
8,Alberta,"10% on the first $131,220 of taxable income, +..."
9,British Columbia,"5.06% on the first $41,725 of taxable income, ..."


In [130]:

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
##import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width',1000)

print('Libraries imported.')

url = 'https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []

for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)

taxtable = []
for row in rows:
    provinces = row[0].text.rstrip()
    taxrates = row[1].text.rstrip()
    taxtable.append([provinces, taxrates])


cols = ['provinces','taxratesCode',]
df_tax = pd.DataFrame(taxtable, columns=cols)
df_tax.loc[df_tax['provinces'].isin(['Ontario','Alberta'])]

Libraries imported.


,provinces,taxratesCode
5,Ontario,"5.05% on the first $44,740 of taxable income, ..."
8,Alberta,"10% on the first $131,220 of taxable income, +..."


In [137]:
url2 = 'https://en.m.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_gross_domestic_product'
result2 = requests.get(url2)
soup2 = BeautifulSoup(result2.text, 'html.parser')
table2 = soup2.find('table',{'class':"wikitable sortable"}).tbody
trs2 = table2.find_all('tr')
rows = []

for tr in trs2:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        



gdptable = []


    
for row2 in rows:
    provinces=row2[0].text.strip()
    gdp = row2[1].text.rstrip()
    national_gdp_contribution = row2[2].text.rstrip()
    population_2018 = row2[3].text.rstrip()
    gdp_per_capital = row2[4].text.rstrip()
    market_income_percapital = row2[5].text.rstrip()
    gdptable.append([provinces,gdp,national_gdp_contribution,population_2018,gdp_per_capital,market_income_percapital])
cols1 = ['provinces','gdp','national_gdp_contribution','population_2018','gdp_per_capital','market_income_percapital']
df_gdp = pd.DataFrame(gdptable, columns=cols1)
df_gdp

,provinces,gdp,national_gdp_contribution,population_2018,gdp_per_capital,market_income_percapital
0,Canada,"2,223,856",100,"37,057,765","60,011","33,029"
1,British Columbia,"295,401",13.28,"5,001,170","59,066","34,426"
2,Alberta,"344,812",15.51,"4,300,721","80,175","39,056"
3,Saskatchewan,"80,679",3.63,"1,162,978","69,373","31,724"
4,Manitoba,"72,688",3.27,"1,353,403","53,708","28,853"
5,Ontario,"857,384",38.55,"14,318,545","59,879","34,033"
6,Quebec,"439,375",19.76,"8,387,632","52,384","29,689"
7,New Brunswick,"36,966",1.66,"770,921","47,950","26,992"
8,Prince Edward Island,"6,994",0.31,"153,584","45,539","27,043"
9,Nova Scotia,"44,354",1.99,"959,500","46,226","28,672"


In [133]:
#table2 = soup2.find('table',{'class':"wikitable sortable"}).tbody
#trs2 = table2.find_all('tr')

#columns11=[v.text.replace('\n','') for v in trs2[0].find_all('th')]
#dfo=pd.DataFrame(columns=columns11)
#for z in range(1,len(trs2)):
 #   tdo=trs2[z].find_all('td')
  #  values=tdo[0].text.replace('\xa0\xa0',''),tdo[1].text,tdo[2].text,tdo[3].text,tdo[4].text,tdo[5].text.replace('\n','')
    
#dfo=dfo.append(pd.Series(values,index=columns11), ignore_index=True)
#dfo
    